Original code taken from https://medium.com/@aungkyawmyint_26195/multi-layer-perceptron-mnist-pytorch-463f795b897a

New things:
* Used a binary transformer with thresholding
* added architecture for binary MLP similar to the one of mpyc

Other
* replaced some magic numbers with config variables
* Added variable to switch between architectures

# Settings

In [1]:
import torch
import numpy as np
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
from config import *
from models.model_conf_mnist_binary import *

NameError: name 'MODEL_DIR' is not defined

# Data Preparation

In [ ]:
# choose the training and testing datasets
train_data = datasets.MNIST(root = 'data', train = True, download = True, transform = transform)
test_data = datasets.MNIST(root = 'data', train = False, download = True, transform = transform)

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_index, valid_index = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_index)
valid_sampler = SubsetRandomSampler(valid_index)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, sampler = train_sampler, num_workers = num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, sampler = valid_sampler, num_workers = num_workers)
test_loader =  torch.utils.data.DataLoader(test_data, batch_size = batch_size, num_workers = num_workers)

# Visualize a batch of training data

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
    
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()
# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    # print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(labels[idx].item()))

# Define Network Architecture

In [ ]:
# Custom Activation functions
# TODO: use binary sign
def sgn(input):
    output = torch.sign(input)
    return output

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()

        if ARCHITECTURE == "relu":
            # linear layer (784 -> hidden_1)
            self.fc1 = nn.Linear(MNIST_PIXEL_CNT, 512)
            # linear layer (n_hidden -> hidden_2)
            self.fc2 = nn.Linear(512,512)
            # linear layer (n_hidden -> 10)
            self.fc3 = nn.Linear(512,10)
            # dropout layer (p=0.2)
            # dropout prevents overfitting of data
            self.droput = nn.Dropout(0.2)

        elif ARCHITECTURE == "binary_mnist":
             self.fc1 = nn.Linear(MNIST_PIXEL_CNT, MNIST_PIXEL_CNT)
             self.fc2 = nn.Linear(MNIST_PIXEL_CNT, MNIST_PIXEL_CNT)
             self.fc3 = nn.Linear(MNIST_PIXEL_CNT, MNIST_PIXEL_CNT)
             self.fc4 = nn.Linear(MNIST_PIXEL_CNT, MNIST_PIXEL_CNT)
             self.fc5 = nn.Linear(MNIST_PIXEL_CNT, 10)

        else:
            raise(ValueError("Invalid architecture!"))
        

    def forward(self, x):
        if ARCHITECTURE == "relu":
            # flatten image input
            x = x.view(-1, MNIST_PIXEL_CNT)
            # add hidden layer, with relu activation function
            x = F.relu(self.fc1(x))
            # add dropout layer
            x = self.droput(x)
            # add hidden layer, with relu activation function
            x = F.relu(self.fc2(x))
            # add dropout layer
            x = self.droput(x)
            # add output layer
            x = self.fc3(x)
            return x

        elif ARCHITECTURE == "binary_mnist":            
            # Pytorch does not support a (binary) sign activation fucntion
            # flatten image input
            x = x.view(-1, MNIST_PIXEL_CNT)

            # add layers
            x = sgn(self.fc1(x))
            x = sgn(self.fc2(x))
            x = sgn(self.fc3(x))
            x = sgn(self.fc4(x))
            x = self.fc5(x)
            return x

        else:
            raise(ValueError("Invalid architecture!"))

# initialize the NN
model = Net()
print(model)

# Specify Loss function and Optimizer

In [ ]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)

# Train the network

In [ ]:
# initialize tracker for minimum validation loss
valid_loss_min = np.Inf  # set initial "min" to infinity
for epoch in range(n_epochs):
    # monitor losses
    train_loss = 0
    valid_loss = 0
    
     
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    for data,label in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output,label)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item() * data.size(0)
        
        
    ######################    
    # validate the model #
    ######################
    model.eval()  # prep model for evaluation
    for data,label in valid_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output,label)
        # update running validation loss 
        valid_loss = loss.item() * data.size(0)
    
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), model_file_name)
        valid_loss_min = valid_loss

# Load the Model with Lowest Validation Loss

In [ ]:
model.load_state_dict(torch.load(model_file_name)

# Test the trained Network

In [ ]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
model.eval() # prep model for evaluation
for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(len(target)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
# calculate and print avg test loss
test_loss = test_loss/len(test_loader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

# Visualize Sample Test Results

In [ ]:
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()
# get sample outputs
output = model(images)
# convert output probabilities to predicted class
_, preds = torch.max(output, 1)
# prep images for display
images = images.numpy()
# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    ax.set_title("{} ({})".format(str(preds[idx].item()), str(labels[idx].item())),
                 color=("green" if preds[idx]==labels[idx] else "red"))